In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Input, BatchNormalization
from keras.models import Model
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
%matplotlib inline

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [6]:
path = kagglehub.dataset_download("sudalairajkumar/daily-temperature-of-major-cities")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/daily-temperature-of-major-cities


# **1. Data Preprocessing**

In [7]:
data = pd.read_csv(os.path.join(path, 'city_temperature.csv'))
data

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [8]:
data.isnull().sum()

Region                  0
Country                 0
State             1450990
City                    0
Month                   0
Day                     0
Year                    0
AvgTemperature          0
dtype: int64

In [9]:
data.drop(columns=['State', 'AvgTemperature', 'Region'], inplace = True)

In [10]:
data

,Country,City,Month,Day,Year
0,Algeria,Algiers,1,1,1995
1,Algeria,Algiers,1,2,1995
2,Algeria,Algiers,1,3,1995
3,Algeria,Algiers,1,4,1995
4,Algeria,Algiers,1,5,1995
...,...,...,...,...,...
2906322,US,San Juan Puerto Rico,7,27,2013
2906323,US,San Juan Puerto Rico,7,28,2013
2906324,US,San Juan Puerto Rico,7,29,2013
2906325,US,San Juan Puerto Rico,7,30,2013


In [11]:
df = data[data['Country'].isin(['Canada', 'Japan', 'Germany', 'France'])]
df

,Country,City,Month,Day,Year
358851,Japan,Osaka,1,1,1995
358852,Japan,Osaka,1,2,1995
358853,Japan,Osaka,1,3,1995
358854,Japan,Osaka,1,4,1995
358855,Japan,Osaka,1,5,1995
...,...,...,...,...,...
1204356,Canada,Winnipeg,6,6,2014
1204357,Canada,Winnipeg,6,7,2014
1204358,Canada,Winnipeg,6,8,2014
1204359,Canada,Winnipeg,6,9,2014


In [13]:
df[df['Country'].isin(['Canada', 'Japan', 'Germany', 'France'])]['City'].unique()

array(['Osaka', 'Sapporo', 'Tokyo', 'Paris', 'Bordeaux', 'Bonn',
       'Frankfurt', 'Hamburg', 'Munich', 'Calgary', 'Edmonton', 'Halifax',
       'Montreal', 'Ottawa', 'Quebec', 'Regina', 'Toronto', 'Vancouver',
       'Winnipeg'], dtype=object)

In [ ]:
def assign_latitude(country):
    if country == 'Canada':
        return 56.130366
    elif country == 'Japan':
        return 36.2048
    elif country == 'Germany':
        return 51.165691
    elif country == 'France':
        return 46.227638


def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'


def random_temp(season, latitude):
    if latitude > 53:  # High latitude (close to North Pole)
        if season == 'Winter':
            return np.random.randint(-20, 5)  # Very cold
        elif season == 'Spring':
            return np.random.randint(-15, 10)  # Cold
        elif season == 'Summer':
            return np.random.randint(5, 15)  # Cool
        elif season == 'Autumn':
            return np.random.randint(-5, 10)  # Cold to cool
            
    elif 47 <= latitude < 53:  # Mid latitude
        if season == 'Winter':
            return np.random.randint(-6, 10)  # Cold
        elif season == 'Spring':
            return np.random.randint(5, 15)  # Mild
        elif season == 'Summer':
            return np.random.randint(15, 25)  # Warm
        elif season == 'Autumn':
            return np.random.randint(7, 18)  # Mild to cool
            
    elif 43 <= latitude < 47:
        if season == 'Winter':
            return np.random.randint(-5, 13)  # Warm
        elif season == 'Spring':
            return np.random.randint(5, 20)  # Warm to hot
        elif season == 'Summer':
            return np.random.randint(10, 25)  # Hot
        elif season == 'Autumn':
            return np.random.randint(5, 20)  # Warm to hot

    else:  # Low latitude (close to Equator)
        if season == 'Winter':
            return np.random.randint(10, 24)  # Warm
        elif season == 'Spring':
            return np.random.randint(18, 28)  # Warm to hot
        elif season == 'Summer':
            return np.random.randint(25, 40)  # Hot
        elif season == 'Autumn':
            return np.random.randint(18, 28)  # Warm to hot


def random_pressure(season, temperature):

    if season == 'Winter':
        # Higher pressure in winter, but slightly lower if temperature is very cold
        if temperature < -10:
            return np.random.randint(1005, 1015)  # Slightly lower pressure for very cold temperatures
        else:
            return np.random.randint(1010, 1020)  # Higher pressure for typical winter temperatures
    elif season == 'Spring':
        # Moderate pressure, adjusted based on temperature
        if temperature < 10:
            return np.random.randint(1005, 1015)  # Slightly higher pressure for cooler spring temperatures
        else:
            return np.random.randint(1000, 1010)  # Lower pressure for warmer spring temperatures
    elif season == 'Summer':
        # Lower pressure in summer, adjusted based on temperature
        if temperature > 30:
            return np.random.randint(1000, 1010)  # Lower pressure for very hot temperatures
        else:
            return np.random.randint(1005, 1015)  # Moderate pressure for typical summer temperatures
    elif season == 'Autumn':
        # Moderate pressure, adjusted based on temperature
        if temperature < 15:
            return np.random.randint(1005, 1015)  # Slightly higher pressure for cooler autumn temperatures
        else:
            return np.random.randint(1000, 1010)  # Lower pressure for warmer autumn temperatures


def random_humidity(season, temperature):

    if season == 'Winter':
        # Lower humidity in winter, especially in very cold conditions
        if temperature < -10:
            return np.random.randint(5, 10)  # Very cold winter has lower humidity
        else:
            return np.random.randint(8, 15)  # Typical winter humidity range

    elif season == 'Spring':
        # Moderate humidity, slightly lower in cool temperatures
        if temperature < 10:
            return np.random.randint(8, 15)  # Cool spring
        else:
            return np.random.randint(10, 18)  # Warmer spring

    elif season == 'Summer':
        # Higher humidity, more pronounced in hotter conditions
        if temperature > 30:
            return np.random.randint(15, 25)  # Very hot summer has higher humidity
        else:
            return np.random.randint(12, 20)  # Moderate summer humidity

    elif season == 'Autumn':
        # Moderate humidity, lower in cooler temperatures
        if temperature < 15:
            return np.random.randint(8, 15)  # Cool autumn
        else:
            return np.random.randint(10, 18)  # Warmer autumn

In [ ]:
df['Latitude'] = tqdm(df['Country'].apply(assign_latitude))

df['Season'] = tqdm(df['Month'].apply(get_season))

df['Temperature'] = tqdm(df[['Season', 'Latitude']].apply(lambda row: random_temp(row['Season'], row['Latitude']), axis=1))   # Temoerature in Celisius

df['Air Pressure'] = tqdm(df[['Season', 'Temperature']].apply(lambda row: random_pressure(row['Season'], row['Temperature']), axis=1))      # Air Pressure in HectoPascal

df['Humidity'] = tqdm(df[['Season', 'Temperature']].apply(lambda row: random_humidity(row['Season'], row['Temperature']), axis=1))      # Humidity in g/m³

In [ ]:
df

In [ ]:
df

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
# Encode categorical variables
country_encoder = LabelEncoder()
city_encoder = LabelEncoder()
season_encoder = LabelEncoder()



df['Country'] = country_encoder.fit_transform(df['Country'])
df['City'] = city_encoder.fit_transform(df['City'])
df['Season'] = season_encoder.fit_transform(df['Season'])


# Save the encoders
joblib.dump(country_encoder, 'country_encoder.pkl')
joblib.dump(city_encoder, 'city_encoder.pkl')
joblib.dump(season_encoder, 'season_encoder.pkl')

In [ ]:
df

In [ ]:
# Plot temperature trends over time
plt.figure(figsize=(12, 6))
sns.lineplot(x='Year', y='Temperature', data=df, errorbar=None)
plt.title('Temperature Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Temperature')
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Boxplot of temperature by season
plt.figure(figsize=(10, 6))
sns.boxplot(x='Season', y='Temperature', data=df)
plt.title('Temperature Distribution by Season')
plt.xlabel('Season')
plt.ylabel('Average Temperature')
plt.legend()
plt.show()

In [ ]:

# Normalize numerical features
scaler = StandardScaler()
df[[ 'Air Pressure', 'Humidity']] = scaler.fit_transform(df[[ 'Air Pressure', 'Humidity']])

# scaler2 = StandardScaler()
# df['Temperature'] = scaler2.fit_transform(df[['Temperature']])

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
df

In [ ]:
# Sort data by City and Date (Year, Month, Day)
df = df.sort_values(by=['City', 'Year', 'Month', 'Day'])

In [ ]:
# Define timesteps (10 days)
timesteps = 10

X = df[['Country', 'City', 'Month', 'Day', 'Year', 'Air Pressure', 'Humidity', 'Season']]
y = df['Temperature']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for time-series models (LSTM, GRU, TCN)
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Print shapes
print(f"X_train shape: {X_train.shape}")  # (samples, timesteps, features)
print(f"y_train shape: {y_train.shape}")  # (samples,)
print(f"X_test shape: {X_test.shape}")    # (samples, timesteps, features)
print(f"y_test shape: {y_test.shape}")    # (samples,)




# **3. Build and Compare GRU, LSTM, and TCN Models**

In [ ]:
# Define a GRU model
input_shape = (X_train.shape[1], 1)

def build_gru_model(input_shape):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.4),
        GRU(32),
        Dropout(0.4),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Define an LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.4),
        LSTM(32),
        Dropout(0.4),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Define a TCN model
def build_tcn_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Temporal Convolutional Layers
    x = Conv1D(64, kernel_size=3, padding='causal', activation='relu')(inputs)
    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(32, kernel_size=3, padding='causal', activation='relu')(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Flatten()(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model



# Build and train the models
gru_model = build_gru_model(input_shape)
lstm_model = build_lstm_model(input_shape)
tcn_model = build_tcn_model(input_shape)

# **4. Train the Models**

In [ ]:
# Train GRU model
gru_history = gru_model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=8, restore_best_weights=True)]
)



In [ ]:
# Train LSTM model
lstm_history = lstm_model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=8, restore_best_weights=True)]
)

In [ ]:
# Train TCN model
tcn_history = tcn_model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

# **5. Evaluating and Testing the Models**

In [ ]:
# Evaluate GRU Model
gru_loss = gru_model.evaluate(X_test, y_test)
print(f'GRU Model Test Loss: {gru_loss}')

# Evaluate LSTM Model
lstm_loss = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Model Test Loss: {lstm_loss}')

# Evaluate TCN Model
tcn_loss = tcn_model.evaluate(X_test, y_test)
print(f'TCN Model Test Loss: {tcn_loss}')

In [ ]:
# Evaluate the models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

# Evaluate GRU
gru_mse, gru_r2 = evaluate_model(gru_model, X_test, y_test)
print(f'GRU - MSE: {gru_mse}, R2: {gru_r2}')

# Evaluate LSTM
lstm_mse, lstm_r2 = evaluate_model(lstm_model, X_test, y_test)
print(f'LSTM - MSE: {lstm_mse}, R2: {lstm_r2}')

# Evaluate TCN
tcn_mse, tcn_r2 = evaluate_model(tcn_model, X_test, y_test)
print(f'TCN - MSE: {tcn_mse}, R2: {tcn_r2}')

In [ ]:
def plot_history(history, title, metrics=None):
    """
    Plots the training and validation loss and metrics over epochs.

    Parameters:
        history: A Keras History object returned by model.fit().
        metrics: List of metrics to plot (e.g., ['accuracy']). Default is None.
    """
    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation metrics (if provided)
    if metrics:
        plt.subplot(1, 2, 2)
        for metric in metrics:
            plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
            plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.title('Training and Validation Metrics')
        plt.xlabel('Epochs')
        plt.ylabel('Metric Value')
        plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(gru_history, 'Training and Validation Loss for GRU')
plot_history(lstm_history, 'Training and Validation Loss for LSTM')
plot_history(tcn_history, 'Training and Validation Loss for TCN')

In [ ]:
# Plot training and validation loss for all models
plt.figure(figsize=(12, 6))
plt.plot(gru_history.history['loss'], label='GRU Training Loss')
plt.plot(gru_history.history['val_loss'], label='GRU Validation Loss')
plt.plot(lstm_history.history['loss'], label='LSTM Training Loss')
plt.plot(lstm_history.history['val_loss'], label='LSTM Validation Loss')
plt.plot(tcn_history.history['loss'], label='TCN Training Loss')
plt.plot(tcn_history.history['val_loss'], label='TCN Validation Loss')
plt.title('Training and Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# **7. Deployment**  (to be continoued)

In [ ]:
# Save the trained model
gru_model.save('GRU-model.h5')
lstm_model.save('LSTM-model.h5')
tcn_model.save('TCN-model.h5')

# Load the model
from tensorflow.keras.models import load_model
loaded_model = load_model('GRU-model.h5')

# Example prediction
sample_input = X_test[0].reshape(1, -1)
prediction = loaded_model.predict(sample_input)
print(f'Predicted Temperature: {prediction[0][0]}')